# Recession Resilience

This project examined whether university towns have their mean housing prices less effected by recessions. The raw date are stored in three different files. 

* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in quarterly intervals is in the file ```gdplev.xls```. 

In [187]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [188]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [189]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. 
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.'''

    university_towns = []
    with open('university_towns.txt','r') as file:
        for line in file:
            if line.strip().endswith('[edit]'):
                state = line.split('[')[0]
            else:
                university_towns.append([state, line.split(' (')[0]])
    university_towns_df = pd.DataFrame(university_towns, columns = ["State", "RegionName"])
    university_towns_df['UniversityTowns'] = 1
    return university_towns_df
get_list_of_university_towns()

,State,RegionName,UniversityTowns
0,Alabama,Auburn,1
1,Alabama,Florence,1
2,Alabama,Jacksonville,1
3,Alabama,Livingston,1
4,Alabama,Montevallo,1
5,Alabama,Troy,1
6,Alabama,Tuscaloosa,1
7,Alabama,Tuskegee,1
8,Alaska,Fairbanks,1
9,Arizona,Flagstaff,1


In [74]:
# Load the GDP data and calculate the GDP change from quarter to quarter
gdp = pd.read_excel('gdplev.xls',skiprows = 5)[214:280][['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']] \
    .rename(columns = {'Unnamed: 4':'Quarter', 'GDP in billions of chained 2009 dollars.1':'GDP'}) \
    .set_index('Quarter')
gdp['Change'] = gdp['GDP'].diff()
gdp

,GDP,Change
Quarter,,
2000q1,12359.1,NaN
2000q2,12592.5,233.4
2000q3,12607.7,15.2
2000q4,12679.3,71.6
2001q1,12643.3,-36.0
2001q2,12710.3,67.0
2001q3,12670.1,-40.2
2001q4,12705.3,35.2
2002q1,12822.3,117.0


Definitions:
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.

In [83]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time '''
    for i in range(len(gdp)-1):
        if (gdp.iloc[i]['Change'] < 0) & (gdp.iloc[i+1]['Change'] < 0):
            return gdp.iloc[i].name
get_recession_start()

'2008q3'

In [84]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time '''
    
    for i in range(1,len(gdp)):
        if (gdp.iloc[i].name > get_recession_start()) \
            & (gdp.iloc[i-1]['Change'] > 0) \
            & (gdp.iloc[i]['Change'] > 0):
            return gdp.iloc[i].name     
get_recession_end()

'2009q4'

In [93]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time '''
    return gdp['GDP'].loc[get_recession_start():get_recession_end()].idxmin()
get_recession_bottom()

'2009q2'

In [153]:
# Load the housing data 
house = pd.read_csv('City_Zhvi_AllHomes.csv') \
        .drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis = 1) 
house['State'] = house['State'].apply(lambda x: states[x])
house = house.set_index(['State','RegionName']) \
            .loc[:,'2000-01':] 
        
house

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
Nevada,Las Vegas,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
California,San Diego,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
Texas,Dallas,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
California,San Jose,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700


In [154]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. '''
    housing_data_to_quarters = house.groupby(pd.PeriodIndex(house.columns,freq = 'q'), axis =1).mean()
    return housing_data_to_quarters
convert_housing_data_to_quarters()

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [190]:
def run_ttest():
    '''Create new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then run a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The value for better is 
    "university town" or "non-university town" depending on which has a lower 
    mean price ratio.'''
    start = get_recession_start().upper()
    end = get_recession_bottom().upper()
    housing_data = convert_housing_data_to_quarters()
    university_towns_df = get_list_of_university_towns()
    
    housing_price_change = (housing_data[end]-housing_data[start])/housing_data[start]
    housing_price_change_df = pd.DataFrame(housing_price_change, columns = ['HousingPriceChange']) \
                                .reset_index() \
                                .dropna()

    merge = pd.merge(housing_price_change_df, university_towns_df, how = 'left', left_on = ['State','RegionName'], right_on = ['State','RegionName']) 
    university_town = merge[merge['UniversityTowns'] == 1.0]['HousingPriceChange']
    non_university_town = merge[merge['UniversityTowns'] != 1.0]['HousingPriceChange']
    
    t,p = ttest_ind(university_town, non_university_town)
    better = 'university town' if university_town.mean() > non_university_town.mean() else 'non-university town'                
    return (p<0.01, p, better)
run_ttest()

(True, 0.008975587540712617, 'university town')